## Loading neccessary libraries

In [1]:
import torch
from torch.utils.data import Dataset
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
# from transformers import AutoTokenizer, BertTokenizer, DistilBertModel
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          AutoConfig,
                          TrainingArguments,
                          Trainer,
                          DataCollatorWithPadding)



# torch.set_grad_enabled(False)

# # Store the model we want to use
# Bert_case = 'bert-base-cased'
# Distilbert_case = 'distilbert-base-cased'

# # We need to create the model and tokenizer
# #bert = AutoModel.from_pretrained(Bert_case)
# bert_distil = DistilBertModel.from_pretrained(Distilbert_case)
# tokenizer = AutoTokenizer.from_pretrained(Distilbert_case)

# bert_distil = DistilBertModel.from_pretrained(Distilbert_case)
# input_pt = tokenizer(
#     'This is a sample input to demonstrate performance of distiled models especially inference time', 
#     return_tensors="pt"
# )


# %time _ = bert_distil(input_pt['input_ids'])
# %time _ = model_pt(input_pt['input_ids'])

## Loading the data

In [18]:
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

df = pd.read_csv(url_data + 'new_df.csv')

y = pd.read_csv(url_data+'y_data.csv')
X = pd.read_csv(url_data + 'X_data.csv')
X_text = df.pop('Processed')
X_text.to_csv('X_text.csv', index=False)

HTTPError: HTTP Error 404: Not Found

In [3]:
y = pd.read_csv(url_data+'y_data.csv')
X = pd.read_csv(url_data + 'X_data.csv')


# y = y.to_numpy()

# Number of unique classes
num_classes = len(y['Conservative'].unique())

# Create id2label dictionary from unique values in the 'label' column
id2label = dict(enumerate(y['Liberal'].unique()))

# Reverse the dictionary to create label2id
label2id = {v: k for k, v in id2label.items()}

print("id2label:", id2label)
print("label2id:", label2id)

id2label: {0: 1, 1: 0}
label2id: {1: 0, 0: 1}


In [4]:
text_train, text_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    stratify=y,
    random_state=42)

In [8]:
# load tokenizer of pretrained model
# here we will use BERT, but we can use any model
model_name = 'bert-base-uncased'
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
tokens_pt = tokenizer("This is an input example", return_tensors="pt")
for key, value in tokens_pt.items():
    print("{}:\n\t{}".format(key, value))

input_ids:
	tensor([[ 101, 2023, 2003, 2019, 7953, 2742,  102]])
token_type_ids:
	tensor([[0, 0, 0, 0, 0, 0, 0]])
attention_mask:
	tensor([[1, 1, 1, 1, 1, 1, 1]])


In [11]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer=None, config=None):
        # Check if the input is already tokenized
        if tokenizer is not None and config is not None:
            # Tokenize the text if a tokenizer and config are provided
            self.encodings = tokenizer(
                texts,
                max_length=config.get('max_length', 128),  # Provide a default max_length if not specified
                truncation=True,
                padding='max_length'  # Ensures all texts are padded to the same length
            )
        else:
            # Assume texts are already tokenized if no tokenizer is provided
            self.encodings = texts
        
        self.labels = labels

    def __getitem__(self, idx):
        # Retrieve the encoded data and corresponding label
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Ensure label is a long tensor
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Convert labels to integers if they're not already
label_dict = {'liberal': 0, 'conservative': 1}
train_labels = [label_dict[label] for label in train_labels]
val_labels = [label_dict[label] for label in val_labels]

train_dataset = RedditDataset(train_encodings, train_labels)
val_dataset = RedditDataset(val_encodings, val_labels)

In [9]:
from torch.optim import AdamW
from transformers import get_scheduler
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

NameError: name 'train_dataset' is not defined